<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/ngram%2Broberta%2Bcnn_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP group project/data')

In [ ]:
import csv
import pandas as pd 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import torch.nn as nn

In [ ]:
import pandas as pd
df_nw= pd.read_csv("final_task3_data.csv")
print(df_nw)

       Unnamed: 0                                               text  label
0               0  CMV: Fat Acceptance Tik Tokers and influencers...      0
1               1  CMV: It makes sense to compare apples and oranges      0
2               2  CMV: Dismissing an individual's experience bec...      0
3               3  CMV: If pro-anorexia groups, suicide promotion...      0
4               4  CMV: Asian Americans generally prospering in t...      0
...           ...                                                ...    ...
12336        1061  how much of your body is your own? if i'm not ...      1
12337        1062  how do you keep a space station clean? the ast...      1
12338        1063  the city where you pay a year's rent up front,...      1
12339        1064  the bbc app gives you the best of bbc wherever...      1
12340        1065  learn how the bbc is working to strengthen tru...      1

[12341 rows x 3 columns]


In [ ]:
df = df_nw

In [ ]:

df_nw['text'].fillna('', inplace=True)

import pandas as pd

# Create a DataFrame
# Rename column 'B' to 'new_name'
df = df.rename(columns={'Generation': 'text'})

print(df)



       Unnamed: 0                                               text  label
0               0  CMV: Fat Acceptance Tik Tokers and influencers...      0
1               1  CMV: It makes sense to compare apples and oranges      0
2               2  CMV: Dismissing an individual's experience bec...      0
3               3  CMV: If pro-anorexia groups, suicide promotion...      0
4               4  CMV: Asian Americans generally prospering in t...      0
...           ...                                                ...    ...
12336        1061  how much of your body is your own? if i'm not ...      1
12337        1062  how do you keep a space station clean? the ast...      1
12338        1063  the city where you pay a year's rent up front,...      1
12339        1064  the bbc app gives you the best of bbc wherever...      1
12340        1065  learn how the bbc is working to strengthen tru...      1

[12341 rows x 3 columns]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Define the n-gram size
n = 3

# Define the number of classes
num_classes = 2


In [ ]:
# Define the PyTorch device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the data frame
# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)



In [ ]:
# Define a tokenizer function that converts text to n-gram word embeddings
def tokenize(text):
    tokens = []
    for i in range(len(text) - n + 1):
        tokens.append(text[i:i+n])
    return tokens


In [ ]:
# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.iloc[idx]["text"]
        label = self.df.iloc[idx]["label"]
        tokenized_text = tokenize(text)
        return tokenized_text, label

In [ ]:
# Define the combined Roberta and CNN model
class RobertaCNN(nn.Module):
    def __init__(self):
        super(RobertaCNN, self).__init__()
        self.roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
        self.cnn = nn.Conv1d(in_channels=1024, out_channels=64, kernel_size=3, padding=1)
        self.fc = nn.Linear(64, num_classes)
        
    def forward(self, tokens):
        # Convert tokens to Roberta embeddings
        embeddings = self.roberta.extract_features(tokens)
        # Convert embeddings to CNN input format
        embeddings = embeddings.transpose(1, 2)
        # Apply CNN layer
        cnn_output = self.cnn(embeddings)
        # Apply ReLU activation function
        cnn_output = F.relu(cnn_output)
        # Flatten the output
        cnn_output = cnn_output.view(cnn_output.size(0), -1)
        # Apply linear layer
        output = self.fc(cnn_output)
        return output

In [ ]:
# Define the training function
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    correct = 0.0
    total = 0.0
    for batch_idx, (tokens, labels) in enumerate(train_loader):
        tokens = tokens.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(tokens)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total
    avg_loss = total_loss / len(train_loader)
    return avg_loss, accuracy

In [ ]:
# Define the validation function
def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for batch_idx, (tokens, labels) in enumerate(val_loader):
            tokens = tokens.to(device)
            labels = labels.to(device)
            outputs = model(tokens)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    avg_loss = total_loss / len(val_loader)
    return avg_loss, accuracy


In [ ]:
# Define the test function
def test(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch_idx, (tokens, labels) in enumerate(test_loader):
            tokens = tokens.to(device)
            labels = labels.to(device)
            outputs = model(tokens)
            _, predicted = torch.max(outputs.data, 1)
            y_true += labels.cpu().numpy().tolist()
            y_pred += predicted.cpu().numpy().tolist()
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    return accuracy, precision, recall, f1


In [ ]:
# Evaluate the model on the test set
test_accuracy, test_precision, test_recall, test_f1 = test(model, test_loader)

# Print the metrics
print("Test Accuracy: {:.4f}".format(test_accuracy))
print("Test Precision: {:.4f}".format(test_precision))
print("Test Recall: {:.4f}".format(test_recall))
print("Test F1 Score: {:.4f}".format(test_f1))


In [ ]:
import praw
import pandas as pd
import torch
from torch.utils.data import DataLoader

# Initialize the Reddit API client
reddit = praw.Reddit(client_id='YOUR_CLIENT_ID', client_secret='YOUR_CLIENT_SECRET', user_agent='YOUR_USER_AGENT')

# Define a function to extract comments from a subreddit
def get_comments_from_subreddit(subreddit, num_comments):
    comments = []
    for comment in reddit.subreddit(subreddit).comments(limit=num_comments):
        comments.append(comment.body)
    return comments


In [ ]:
# Extract comments from a subreddit
df_reddit = pd.read_csv('reddit_data.csv')  # Load Reddit data from a CSV file
comments = df_reddit['text']

# Preprocess the comments
preprocessed_comments = preprocess_comments(comments)

# Convert the preprocessed comments into tensors
tokens = [torch.tensor([word_to_idx.get(word, 0) for word in comment]) for comment in preprocessed_comments]
labels = torch.zeros(len(tokens), dtype=torch.long)  # Set the labels to 0, since we're not interested in sentiment polarity



In [ ]:

# Create a test dataset and loader
test_dataset = TextDataset(tokens, labels)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Evaluate the model on the test set
test_accuracy, test_precision, test_recall, test_f1 = test(model, test_loader)



In [ ]:
# Print the metrics
print("Test Accuracy: {:.4f}".format(test_accuracy))
print("Test Precision: {:.4f}".format(test_precision))
print("Test Recall: {:.4f}".format(test_recall))
print("Test F1 Score: {:.4f}".format(test_f1))